<a target="_blank" href="https://colab.research.google.com/github/taobrienlbl/advanced_earth_science_data_analysis/blob/fall_2025_iub/content/lessons/03_data_science_modules/03_winter_temperature_enso.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Work-along: El Nino and IN Winter Temperatures

This notebook examines the relationship between winter minimum temperatures in southern central IN and El Nino.

**Goal:** examine the relationship between El Nino and minimum winter (Dec-Jan-Feb) temperatures in Bloomington, IN

<div style="max-width:720px"><div style="position:relative;padding-bottom:56.25%"><iframe id="kaltura_player" src='https://cdnapisec.kaltura.com/p/1751071/embedPlaykitJs/uiconf_id/55382703?iframeembed=true&amp;entry_id=1_sovpwyf8&amp;config%5Bprovider%5D=%7B%22widgetId%22%3A%221_hcj1au9f%22%7D&amp;config%5Bplayback%5D=%7B%22startTime%22%3A0%7D'  allowfullscreen webkitallowfullscreen mozAllowFullScreen allow="autoplay *; fullscreen *; encrypted-media *" sandbox="allow-downloads allow-forms allow-same-origin allow-scripts allow-top-navigation allow-pointer-lock allow-popups allow-modals allow-orientation-lock allow-popups-to-escape-sandbox allow-presentation allow-top-navigation-by-user-activation" title="EAS-G 690 - Week 3 - ENSO and Tmin" style="position:absolute;top:0;left:0;width:100%;height:100%;border:0"></iframe></div></div>

```{note}
Click [here](https://iu.mediaspace.kaltura.com/media/t/1_sovpwyf8) if the above video has issues playing.
```

**Method:**

 * obtain minimum temperature data from [NOAA NClimDiv](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00005), using the *NCEI Direct Download* method
 * read the dataset using `pandas`, filter out data for southern central Indiana
 * convert the data to timeseries format
 * obtain the [ENSO longitude index](https://cascade.lbl.gov/enso-longitude-index-eli/) Excel file and read with `pandas`; convert to timeseries format
 * make the ENSO and NClimDiv timeseries align (same start and end dates, same period - DJF, etc)
 * plot both timeseries
 * plot a scatter plot
 * do linear regression and correlation analyses

For this solution notebook, I will be using data directly from NOAA instead of downloading it, since `pandas` has the ability to open data from a URL.

```{attention}
If you have difficulty accessing the NOAA data site (which happened during class), you can download the two data files referenced in the video below.
```

 * [climdiv-tmindv-v1.0.0-20250905](https://raw.githubusercontent.com/taobrienlbl/advanced_earth_science_data_analysis/refs/heads/fall_2025_iub/content/lessons/03_data_science_modules/climdiv-tmindv-v1.0.0-20250905)
 * [divisional-readme.txt](https://github.com/taobrienlbl/advanced_earth_science_data_analysis/raw/refs/heads/fall_2025_iub/content/lessons/03_data_science_modules/divisional-readme.txt)

## Part 1: Getting the NClimDiv data (workalong)

```{attention}
Download this notebook and place it in your course folder, running it in Visual Studio Code (not Google Colab).  You'll commit this to your git repository at the end.
```

In [ ]:
""" Import libraries """
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd

In [ ]:
""" Read the tmin data file. """


In [ ]:
""" Filter out south central Indiana. """


In [ ]:
""" Convert the dataset to a timeseries. """


In [ ]:
""" Select and average over winter times (DJF). """

## Part 2: Reading and opening El Nino data

```{attention}
Work in groups on your own from this point forward.
```

In [ ]:
# set the URL to the ELI file
eli_url = "https://portal.nersc.gov/archive/home/projects/cascade/www/ELI/ELI_ERSSTv5_1854.01-2020.05.xlsx"

# read in the data


hint: use the `merge()` method in pandas to get the ELI and NOAA data to align.

In [ ]:
print('ok')